In [1]:
import math
import numpy as np
import os
import pandas as pd
import sys

In [2]:
# choose input file
input_filename = "jabref_pr_output.csv"

# get project name
# To do this, input file must have format <project_name>_<rest of filename>.csv, e.g. jabref_example.csv
proj_name      = input_filename.split( "_" )[0]
print( "project name: " + proj_name )

# choose directory where all input and output data will be written
data_dir       = "/home/j/git/osl/ETL2-Pipeline/data/"
input_dir      = data_dir + "input/"
output_dir     = data_dir + "output/"
proj_out_dir   = output_dir + proj_name + '/'
print( "project output directory: " + proj_out_dir )

# create input file path
input_file = input_dir + input_filename
print( "\ninput file: " + input_file )

# create project-specific output dirs
os.makedirs( proj_out_dir, exist_ok=True )

# choose dir for intermediate outputs
new_fields_dir  = proj_out_dir + "new_fields/"
print( "\nNew fields directory: " + new_fields_dir )

os.makedirs( new_fields_dir, exist_ok=True )

# create newFields output filepath
new_fields_output_format = new_fields_dir + proj_name + "_newFields"

new_fields_csv = new_fields_output_format + ".csv"
new_fields_txt = new_fields_output_format + ".txt"
print( "new fields csv: " + new_fields_csv )
print( "new fields txt: " + new_fields_txt )

# create newFields3 output filepath
new_fields3_csv = new_fields_output_format + "3.csv"
new_fields3_txt = new_fields_output_format + "3.txt"
print( "new fields 3 csv: " + new_fields3_csv )
print( "new fields 3 txt: " + new_fields3_txt )

project name: jabref
project output directory: /home/j/git/osl/ETL2-Pipeline/data/output/jabref/

input file: /home/j/git/osl/ETL2-Pipeline/data/input/jabref_pr_output.csv

New fields directory: /home/j/git/osl/ETL2-Pipeline/data/output/jabref/new_fields/
new fields csv: /home/j/git/osl/ETL2-Pipeline/data/output/jabref/new_fields/jabref_newFields.csv
new fields txt: /home/j/git/osl/ETL2-Pipeline/data/output/jabref/new_fields/jabref_newFields.txt
new fields 3 csv: /home/j/git/osl/ETL2-Pipeline/data/output/jabref/new_fields/jabref_newFields3.csv
new fields 3 txt: /home/j/git/osl/ETL2-Pipeline/data/output/jabref/new_fields/jabref_newFields3.txt


In [3]:
# for the sake of consistency, we want to make sure that the projects are consistently named. Use all lowercase, e.g. presto instead of Presto
mergeNA_output_filename = proj_out_dir + proj_name + "_df_mergeNA.csv"
print( mergeNA_output_filename )

/home/j/git/osl/ETL2-Pipeline/data/output/jabref/jabref_df_mergeNA.csv


In [4]:
merge_output_filename = proj_out_dir + proj_name + "_df_merge.csv"
print( merge_output_filename )

/home/j/git/osl/ETL2-Pipeline/data/output/jabref/jabref_df_merge.csv


In [5]:
# read out the contents of the input file. Change separator if necessary
prIssues = pd.read_csv(input_file, header = 0, sep="\a")

In [6]:
pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 999

In [7]:
prIssues.head(5)

,Issue_Number,Issue_Title,Issue_Author_Name,Issue_Author_Login,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author_Name,PR_Author_Login,PR_Closed_Date,PR_Body,PR_Comments,Commit_Author_Name,Commit_Date,Commit_Message,isPR
0,1,New Sorting/Export preferences,Olaf Lenz,olenz,"03/12/14, 11:38:01 AM","This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,Olaf Lenz,olenz,"03/12/14, 11:38:01 AM","This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Olaf Lenz,"03/12/14, 09:16:45 AM",Merge branch 'sorting',1
1,2,Basic gradle integration,Simon Harrer,simonharrer,"03/12/14, 06:29:22 PM",This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Basic gradle integration,Simon Harrer,simonharrer,"03/12/14, 06:29:22 PM",This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Simon Harrer,"03/12/14, 05:20:08 PM",Gradle build works basically. Reuses existing folder structure.,1
2,3,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,egonw,"03/15/14, 12:55:32 PM",NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,egonw,"03/15/14, 12:55:32 PM",NaN,NaN,Egon Willighagen,"03/15/14, 09:33:10 AM",Some example new Dutch translations via the GitHub web page.,1
3,4,Spanish translation update,Jorge Tornero,jtornero,"03/17/14, 10:20:50 PM",Three new strings translated.,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,Jorge Tornero,jtornero,"03/17/14, 10:20:50 PM",Three new strings translated.,NaN,jtornero,"03/17/14, 09:28:26 PM",Spanish translation update,1
4,5,Update JabRef_in.properties,NaN,was123,"03/18/14, 05:12:53 AM",Indonesian translation added,NaN,Update JabRef_in.properties,NaN,was123,"03/18/14, 05:12:53 AM",Indonesian translation added,NaN,was123,"03/18/14, 12:01:58 AM",Update JabRef_in.properties\n\nIndonesian translation added,1


In [8]:
def clean(issue):
    buffer = ""

    for char in issue:
        if char.isdigit():
            buffer += char
        else:
            break

    print( "cleaned:",buffer )
    return buffer




def searchIssue(text):
    issues = []
    prs = []
    i = 0

    if text == '' or (isinstance(text, float) and  math.isnan(text)):
        print("PR:",prnumber," title is NaN:", text)
        
    else:
        while (i<len(text)):
            print("PR:",prnumber," searching from:",i,"limit:",len(text))
            
            if (prnumber == 2300):
                print ("Analyzing", text)
                
            resultTralha = text.find('#',i)
            
            if (prnumber == 2300):
                print ("tralha position", resultTralha)
                
            if (resultTralha != -1):
                next = resultTralha + 5 #issue number with 4 digits

                if (next < len(text)):
                    num = resultTralha+1
                    numberIssue = text[resultTralha+1:next]
                    print("PR:",prnumber," Issue:",numberIssue)
                    numberIssue = clean(numberIssue)
                    
                    if numberIssue == '' or numberIssue == '333':
                        print("Issue discarded after clean",numberIssue)
                        
                    else:
                        print("Issue added",numberIssue)
                        issues.append(numberIssue)
                        prs.append(prnumber)
                        
                else:
                    next = resultTralha + 4 # issue number < 4
                    if (next < len(text)):
                        num = resultTralha+1
                        numberIssue = text[resultTralha+1:next]
                        print("PR:",prnumber," Issue:",numberIssue)
                        
                        numberIssue = clean(numberIssue)
                        if numberIssue == '' or numberIssue == '333':
                            print("Issue discarded after clean",numberIssue)
                            
                        else:
                            print("Issue added",numberIssue)
                            issues.append(numberIssue)
                            prs.append(prnumber)
                            
                    else:
                        break
            else:
                break
                
            i=resultTralha+1
            
    return issues, prs




globalIssues = []
prsToLink = []
trainPRs = []
testPRs = []

for row in prIssues.itertuples(index=True, name='Pandas'):

    prtitle = row.PR_Title
    prbody = row.PR_Body
    prnumber = row.Issue_Number
    ispr = row.isPR
    
    if (ispr==1):
        resList = []
        prsList = []

        resList, prsList = searchIssue( prtitle )
        
        if len(resList) !=0:

            globalIssues.extend(resList)
            prsToLink.extend(prsList)
            testPRs.extend(prsList)

        resList = []
        prsList = []
        
        #search body
        resList, prsList = searchIssue(prbody)
        if len(resList) !=0:
            
            globalIssues.extend(resList)
            prsToLink.extend(prsList)
            testPRs.extend(prsList)
  
            

PR: 1  searching from: 0 limit: 30
PR: 1  searching from: 0 limit: 201
PR: 2  searching from: 0 limit: 24
PR: 2  searching from: 0 limit: 188
PR: 3  searching from: 0 limit: 60
PR: 3  title is NaN: nan
PR: 4  searching from: 0 limit: 26
PR: 4  searching from: 0 limit: 29
PR: 5  searching from: 0 limit: 27
PR: 5  searching from: 0 limit: 28
PR: 6  searching from: 0 limit: 20
PR: 6  title is NaN: nan
PR: 7  searching from: 0 limit: 56
PR: 7  Issue: 841
cleaned: 841
Issue added 841
PR: 7  searching from: 52 limit: 56
PR: 7  searching from: 0 limit: 941
PR: 8  searching from: 0 limit: 49
PR: 8  searching from: 0 limit: 2307
PR: 8  Issue: 869 
cleaned: 869
Issue added 869
PR: 8  searching from: 2217 limit: 2307
PR: 9  searching from: 0 limit: 34
PR: 9  searching from: 0 limit: 110
PR: 10  searching from: 0 limit: 43
PR: 10  searching from: 0 limit: 482
PR: 11  searching from: 0 limit: 42
PR: 11  Issue: 880 
cleaned: 880
Issue added 880
PR: 11  searching from: 17 limit: 42
PR: 11  searching 

PR: 581  searching from: 0 limit: 162
PR: 582  searching from: 0 limit: 22
PR: 582  searching from: 0 limit: 29
PR: 583  searching from: 0 limit: 60
PR: 583  searching from: 0 limit: 284
PR: 584  searching from: 0 limit: 17
PR: 584  searching from: 0 limit: 453
PR: 585  searching from: 0 limit: 44
PR: 585  searching from: 0 limit: 28
PR: 587  searching from: 0 limit: 30
PR: 587  searching from: 0 limit: 75
PR: 588  searching from: 0 limit: 38
PR: 588  title is NaN: nan
PR: 589  searching from: 0 limit: 26
PR: 589  title is NaN: nan
PR: 590  searching from: 0 limit: 11
PR: 590  title is NaN: nan
PR: 591  searching from: 0 limit: 21
PR: 591  title is NaN: nan
PR: 592  searching from: 0 limit: 32
PR: 592  title is NaN: nan
PR: 594  searching from: 0 limit: 44
PR: 594  Issue: 593 
cleaned: 593
Issue added 593
PR: 594  searching from: 8 limit: 44
PR: 594  searching from: 0 limit: 176
PR: 594  Issue: 593.
cleaned: 593
Issue added 593
PR: 594  searching from: 18 limit: 176
PR: 595  searching 

PR: 1199  searching from: 0 limit: 176
PR: 1200  searching from: 0 limit: 33
PR: 1200  searching from: 0 limit: 189
PR: 1201  searching from: 0 limit: 61
PR: 1201  searching from: 0 limit: 59
PR: 1202  searching from: 0 limit: 46
PR: 1202  searching from: 0 limit: 494
PR: 1205  searching from: 0 limit: 70
PR: 1205  searching from: 0 limit: 840
PR: 1205  Issue: issu
cleaned: 
Issue discarded after clean 
PR: 1205  searching from: 280 limit: 840
PR: 1206  searching from: 0 limit: 69
PR: 1206  searching from: 0 limit: 478
PR: 1206  Issue: ## S
cleaned: 
Issue discarded after clean 
PR: 1206  searching from: 171 limit: 478
PR: 1206  Issue: # St
cleaned: 
Issue discarded after clean 
PR: 1206  searching from: 172 limit: 478
PR: 1206  Issue:  Ste
cleaned: 
Issue discarded after clean 
PR: 1206  searching from: 173 limit: 478
PR: 1207  searching from: 0 limit: 22
PR: 1207  searching from: 0 limit: 1681
PR: 1207  Issue: 1080
cleaned: 1080
Issue added 1080
PR: 1207  searching from: 152 limit: 1

PR: 1698  searching from: 0 limit: 30
PR: 1700  searching from: 0 limit: 14
PR: 1700  searching from: 0 limit: 200
PR: 1701  searching from: 0 limit: 56
PR: 1701  searching from: 0 limit: 240
PR: 1702  searching from: 0 limit: 57
PR: 1702  searching from: 0 limit: 240
PR: 1704  searching from: 0 limit: 58
PR: 1704  searching from: 0 limit: 1023
PR: 1704  Issue: issu
cleaned: 
Issue discarded after clean 
PR: 1704  searching from: 71 limit: 1023
PR: 1705  searching from: 0 limit: 16
PR: 1705  searching from: 0 limit: 631
PR: 1706  searching from: 0 limit: 24
PR: 1706  searching from: 0 limit: 139
PR: 1709  searching from: 0 limit: 51
PR: 1709  searching from: 0 limit: 188
PR: 1710  searching from: 0 limit: 9
PR: 1710  searching from: 0 limit: 631
PR: 1712  searching from: 0 limit: 49
PR: 1712  searching from: 0 limit: 77
PR: 1712  Issue: 1609
cleaned: 1609
Issue added 1609
PR: 1712  searching from: 9 limit: 77
PR: 1714  searching from: 0 limit: 51
PR: 1714  searching from: 0 limit: 603


PR: 2068  searching from: 0 limit: 1574
PR: 2069  searching from: 0 limit: 65
PR: 2069  searching from: 0 limit: 268
PR: 2069  Issue: 1808
cleaned: 1808
Issue added 1808
PR: 2069  searching from: 167 limit: 268
PR: 2070  searching from: 0 limit: 38
PR: 2070  searching from: 0 limit: 201
PR: 2070  Issue: 1704
cleaned: 1704
Issue added 1704
PR: 2070  searching from: 14 limit: 201
PR: 2070  Issue: 2021
cleaned: 2021
Issue added 2021
PR: 2070  searching from: 51 limit: 201
PR: 2071  searching from: 0 limit: 38
PR: 2071  searching from: 0 limit: 269
PR: 2072  searching from: 0 limit: 27
PR: 2072  searching from: 0 limit: 107
PR: 2072  Issue: `-co
cleaned: 
Issue discarded after clean 
PR: 2072  searching from: 75 limit: 107
PR: 2074  searching from: 0 limit: 39
PR: 2074  searching from: 0 limit: 28
PR: 2075  searching from: 0 limit: 28
PR: 2075  searching from: 0 limit: 481
PR: 2075  Issue: 2064
cleaned: 2064
Issue added 2064
PR: 2075  searching from: 157 limit: 481
PR: 2077  searching from

...

PR: 7173  searching from: 106 limit: 2497
PR: 7173  Issue: 6394
cleaned: 6394
Issue added 6394
PR: 7173  searching from: 322 limit: 2497
PR: 7173  Issue: setG
cleaned: 
Issue discarded after clean 
PR: 7173  searching from: 1392 limit: 2497
PR: 7174  searching from: 0 limit: 46
PR: 7174  searching from: 0 limit: 790
PR: 7174  Issue: 5777
cleaned: 5777
Issue added 5777
PR: 7174  searching from: 7 limit: 790
PR: 7178  searching from: 0 limit: 38
PR: 7178  searching from: 0 limit: 1215
PR: 7178  Issue: 7176
cleaned: 7176
Issue added 7176
PR: 7178  searching from: 7 limit: 1215
PR: 7178  Issue: 6944
cleaned: 6944
Issue added 6944
PR: 7178  searching from: 31 limit: 1215
PR: 7178  Issue: 333"
cleaned: 333
Issue discarded after clean 333
PR: 7178  searching from: 312 limit: 1215
PR: 7179  searching from: 0 limit: 39
PR: 7179  Issue: 7126
cleaned: 7126
Issue added 7126
PR: 7179  searching from: 25 limit: 39
PR: 7179  searching from: 0 limit: 636
PR: 7179  Issue: 7126
cleaned: 7126
Issue added

PR: 7329  searching from: 0 limit: 41
PR: 7329  searching from: 0 limit: 4253
PR: 7329  Issue: 394<
cleaned: 394
Issue added 394
PR: 7329  searching from: 846 limit: 4253
PR: 7329  Issue: 394<
cleaned: 394
Issue added 394
PR: 7329  searching from: 1604 limit: 4253
PR: 7329  Issue: abou
cleaned: 
Issue discarded after clean 
PR: 7329  searching from: 3258 limit: 4253
PR: 7329  Issue:  (de
cleaned: 
Issue discarded after clean 
PR: 7329  searching from: 3461 limit: 4253
PR: 7329  Issue:  (de
cleaned: 
Issue discarded after clean 
PR: 7329  searching from: 3498 limit: 4253
PR: 7330  searching from: 0 limit: 37
PR: 7330  searching from: 0 limit: 6936
PR: 7330  Issue:  cod
cleaned: 
Issue discarded after clean 
PR: 7330  searching from: 388 limit: 6936
PR: 7330  Issue:  run
cleaned: 
Issue discarded after clean 
PR: 7330  searching from: 973 limit: 6936
PR: 7330  Issue: 2980
cleaned: 2980
Issue added 2980
PR: 7330  searching from: 1509 limit: 6936
PR: 7330  Issue: 2997
cleaned: 2997
Issue a

Issue discarded after clean 
PR: 7484  searching from: 1770 limit: 2446
PR: 7484  Issue:  Scr
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1771 limit: 2446
PR: 7484  Issue: ## B
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1786 limit: 2446
PR: 7484  Issue: # Be
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1787 limit: 2446
PR: 7484  Issue:  Bef
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1788 limit: 2446
PR: 7484  Issue: ### 
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1806 limit: 2446
PR: 7484  Issue: ## F
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1807 limit: 2446
PR: 7484  Issue: # Fi
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1808 limit: 2446
PR: 7484  Issue:  Fil
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1809 limit: 2446
PR: 7484  Issue: ### 
cleaned: 
Issue discarded after clean 
PR: 7484  searching from: 1966 limi

Issue discarded after clean 333
PR: 7626  searching from: 106 limit: 1064
PR: 7628  searching from: 0 limit: 12
PR: 7628  searching from: 0 limit: 1064
PR: 7628  Issue: 333"
cleaned: 333
Issue discarded after clean 333
PR: 7628  searching from: 106 limit: 1064
PR: 7630  searching from: 0 limit: 23
PR: 7630  searching from: 0 limit: 108
PR: 7636  searching from: 0 limit: 41
PR: 7636  searching from: 0 limit: 1317
PR: 7636  Issue: 7635
cleaned: 7635
Issue added 7635
PR: 7636  searching from: 28 limit: 1317
PR: 7636  Issue: 6207
cleaned: 6207
Issue added 6207
PR: 7636  searching from: 83 limit: 1317
PR: 7638  searching from: 0 limit: 19
PR: 7638  searching from: 0 limit: 1316
PR: 7638  Issue: 333"
cleaned: 333
Issue discarded after clean 333
PR: 7638  searching from: 106 limit: 1316
PR: 7638  Issue: 6207
cleaned: 6207
Issue added 6207
PR: 7638  searching from: 355 limit: 1316
PR: 7639  searching from: 0 limit: 65
PR: 7639  searching from: 0 limit: 2391
PR: 7639  Issue: 5850
cleaned: 5850


Issue added 520
PR: 7779  searching from: 780 limit: 5365
PR: 7779  Issue: 526<
cleaned: 526
Issue added 526
PR: 7779  searching from: 1381 limit: 5365
PR: 7779  Issue: 525<
cleaned: 525
Issue added 525
PR: 7779  searching from: 1681 limit: 5365
PR: 7779  Issue: 524<
cleaned: 524
Issue added 524
PR: 7779  searching from: 2327 limit: 5365
PR: 7779  Issue: 520<
cleaned: 520
Issue added 520
PR: 7779  searching from: 2810 limit: 5365
PR: 7779  Issue: 520<
cleaned: 520
Issue added 520
PR: 7779  searching from: 3333 limit: 5365
PR: 7779  Issue: abou
cleaned: 
Issue discarded after clean 
PR: 7779  searching from: 3851 limit: 5365
PR: 7779  Issue:  (de
cleaned: 
Issue discarded after clean 
PR: 7779  searching from: 4054 limit: 5365
PR: 7779  Issue:  (de
cleaned: 
Issue discarded after clean 
PR: 7779  searching from: 4091 limit: 5365
PR: 7780  searching from: 0 limit: 56
PR: 7780  searching from: 0 limit: 1893
PR: 7780  Issue: abou
cleaned: 
Issue discarded after clean 
PR: 7780  searching f

In [9]:
type(next)

builtin_function_or_method

In [10]:
type(prtitle)

str

In [11]:
testPRs

[7,
 8,
 11,
 12,
 12,
 14,
 14,
 17,
 26,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 36,
 38,
 39,
 42,
 52,
 54,
 137,
 137,
 143,
 143,
 165,
 175,
 175,
 182,
 183,
 187,
 189,
 200,
 204,
 204,
 211,
 211,
 219,
 230,
 258,
 287,
 300,
 300,
 304,
 319,
 320,
 330,
 347,
 352,
 357,
 370,
 391,
 396,
 418,
 429,
 429,
 440,
 452,
 452,
 452,
 452,
 452,
 452,
 452,
 459,
 472,
 476,
 491,
 500,
 515,
 560,
 567,
 568,
 569,
 594,
 594,
 595,
 595,
 600,
 602,
 605,
 605,
 626,
 648,
 660,
 663,
 677,
 677,
 677,
 677,
 683,
 690,
 694,
 694,
 711,
 715,
 716,
 718,
 718,
 718,
 718,
 718,
 739,
 752,
 755,
 793,
 799,
 828,
 842,
 842,
 844,
 853,
 853,
 858,
 864,
 874,
 874,
 877,
 879,
 879,
 884,
 887,
 893,
 893,
 903,
 915,
 925,
 926,
 929,
 937,
 942,
 947,
 949,
 956,
 966,
 966,
 974,
 979,
 988,
 994,
 994,
 996,
 999,
 999,
 999,
 1010,
 1013,
 1014,
 1020,
 1021,
 1041,
 1050,
 1054,
 1056,
 1057,
 1064,
 1079,
 1089,
 1100,
 1100,
 1100,
 1103,
 1116,
 1116,
 1116,
 1116,
 

In [12]:
globalIssues

['841',
 '869',
 '880',
 '959',
 '959',
 '960',
 '960',
 '850',
 '958',
 '762',
 '692',
 '685',
 '602',
 '225',
 '1240',
 '1240',
 '1253',
 '919',
 '1269',
 '876',
 '51',
 '1278',
 '1277',
 '136',
 '128',
 '1285',
 '164',
 '122',
 '122',
 '173',
 '101',
 '114',
 '188',
 '191',
 '318',
 '582',
 '114',
 '186',
 '173',
 '130',
 '244',
 '208',
 '290',
 '290',
 '298',
 '318',
 '301',
 '328',
 '346',
 '366',
 '1191',
 '367',
 '116',
 '370',
 '388',
 '420',
 '420',
 '427',
 '315',
 '384',
 '384',
 '443',
 '445',
 '498',
 '490',
 '458',
 '470',
 '473',
 '314',
 '495',
 '514',
 '116',
 '565',
 '564',
 '116',
 '593',
 '593',
 '545',
 '545',
 '599',
 '601',
 '337',
 '495',
 '608',
 '647',
 '650',
 '627',
 '669',
 '621',
 '669',
 '621',
 '667',
 '410',
 '212',
 '213',
 '520',
 '706',
 '116',
 '621',
 '669',
 '621',
 '669',
 '756',
 '738',
 '751',
 '741',
 '762',
 '789',
 '824',
 '492',
 '492',
 '685',
 '821',
 '60',
 '454',
 '859',
 '856',
 '815',
 '295',
 '466',
 '466',
 '883',
 '507',
 '402',
 '

In [13]:
prsToLink

[7,
 8,
 11,
 12,
 12,
 14,
 14,
 17,
 26,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 36,
 38,
 39,
 42,
 52,
 54,
 137,
 137,
 143,
 143,
 165,
 175,
 175,
 182,
 183,
 187,
 189,
 200,
 204,
 204,
 211,
 211,
 219,
 230,
 258,
 287,
 300,
 300,
 304,
 319,
 320,
 330,
 347,
 352,
 357,
 370,
 391,
 396,
 418,
 429,
 429,
 440,
 452,
 452,
 452,
 452,
 452,
 452,
 452,
 459,
 472,
 476,
 491,
 500,
 515,
 560,
 567,
 568,
 569,
 594,
 594,
 595,
 595,
 600,
 602,
 605,
 605,
 626,
 648,
 660,
 663,
 677,
 677,
 677,
 677,
 683,
 690,
 694,
 694,
 711,
 715,
 716,
 718,
 718,
 718,
 718,
 718,
 739,
 752,
 755,
 793,
 799,
 828,
 842,
 842,
 844,
 853,
 853,
 858,
 864,
 874,
 874,
 877,
 879,
 879,
 884,
 887,
 893,
 893,
 903,
 915,
 925,
 926,
 929,
 937,
 942,
 947,
 949,
 956,
 966,
 966,
 974,
 979,
 988,
 994,
 994,
 996,
 999,
 999,
 999,
 1010,
 1013,
 1014,
 1020,
 1021,
 1041,
 1050,
 1054,
 1056,
 1057,
 1064,
 1079,
 1089,
 1100,
 1100,
 1100,
 1103,
 1116,
 1116,
 1116,
 1116,
 

In [14]:
t = prIssues.loc[11:11,"PR_Title"]
b = prIssues.loc[11:11,"PR_Body"]

In [15]:
pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 9999

In [16]:
t


11    BugFix for #959 "StringIndexOutOfBoundsException with invalid Preview text"
Name: PR_Title, dtype: object

In [17]:
prIssues.loc[prIssues['Issue_Number'] == 1, "PR_Body"]

0    This will add a new "File Sorting" Tab to the preferences that allows to exactly speci...
Name: PR_Body, dtype: object

In [18]:
newdf = prIssues.loc[953]

In [19]:
newdf


Issue_Number                                                                                                956
Issue_Title                                                                Make tabbedPane in mainframe private
Issue_Author_Name                                                                                   Tobias Diez
Issue_Author_Login                                                                                   tobiasdiez
Issue_Closed_Date                                                                         03/14/16, 08:54:08 AM
Issue_Body            Triggered by #931. \n- Makes tabbedPane in mainframe private\n- Set tab tooltips in co...
Issue_Comments                                                                                    :+1: \n =||= 
PR_Title                                                                   Make tabbedPane in mainframe private
PR_Author_Name                                                                                      Tobi

In [20]:
print(os.linesep.join([str(t), str(b)]))

11    BugFix for #959 "StringIndexOutOfBoundsException with invalid Preview text"
Name: PR_Title, dtype: object
11    Fixed #959 "StringIndexOutOfBoundsException with invalid Preview text". java.lang.Stri...
Name: PR_Body, dtype: object


In [21]:
newdf.to_csv(None)

',953\nIssue_Number,956\nIssue_Title,Make tabbedPane in mainframe private\nIssue_Author_Name,Tobias Diez\nIssue_Author_Login,tobiasdiez\nIssue_Closed_Date,"03/14/16, 08:54:08 AM"\nIssue_Body,"Triggered by #931. \n- Makes tabbedPane in mainframe private\n- Set tab tooltips in consistent way (for example they should now also show up after the tab list is sorted)\n## \n- [x] Change in CHANGELOG.md described? not worth it\n- [x] Changes in pull request outlined? (What, why, ...)\n- [x] Tests created for changes? well....\n- [x] Tests green?"\nIssue_Comments,":+1: \n =||= "\nPR_Title,Make tabbedPane in mainframe private\nPR_Author_Name,Tobias Diez\nPR_Author_Login,tobiasdiez\nPR_Closed_Date,"03/14/16, 08:54:08 AM"\nPR_Body,"Triggered by #931. \n- Makes tabbedPane in mainframe private\n- Set tab tooltips in consistent way (for example they should now also show up after the tab list is sorted)\n## \n- [x] Change in CHANGELOG.md described? not worth it\n- [x] Changes in pull request outlined? 

In [22]:

i=0
line = []
dataLine = ""
dataLine = "PR, Issue, Issue_Title_Linked, Issue_Body_Linked, Issue_Comment_Linked, isTrain"
line.append((dataLine)) 
pr_ant = 0
issue_ant = 0

headerdf = ['PR', 'Issue', 'Issue_Title_Linked', 'Issue_Body_Linked', 'Issue_Comment_Linked', 'isTrain']

tup_line = []

while i < len(globalIssues):
#for issue in globalIssues: 
    issue = globalIssues[i]
    print ("processing...",issue)
    pr = prsToLink[i]
    #ispr = prIssues.loc[prIssues['PR_Number'] == issue, 'isPR'].values
    ispr = prIssues.loc[prIssues['Issue_Number'] == int(issue), 'isPR'].values
    #print(ispr)
    #df.query('PR_Number==3')['A']
    if (ispr==0) :
        if (issue!=issue_ant or pr!=pr_ant):
            issuetitle = prIssues.loc[prIssues['Issue_Number'] == int(issue), 'Issue_Title'].values
            issuebody = prIssues.loc[prIssues['Issue_Number'] == int(issue), 'Issue_Body'].values
            issuecomments = prIssues.loc[prIssues['Issue_Number'] == int(issue), 'Issue_Comments'].values
            isTrain = 0
            print("Issue:",issue, " linked from PR:", pr)

            dataLine =  str(pr) + "," + str(issue) + "," + str(issuetitle[0]) + "," + str(issuebody[0]) + "," + str(issuecomments[0]) + "," + str(isTrain)
            line.append((dataLine))
            #print(line)
            tup = ( str(pr) , str(issue) , str(issuetitle[0]) , str(issuebody[0]) , str(issuecomments[0]) , str(isTrain))
            list_tup = [ str(pr) , str(issue) , str(issuetitle[0]) , str(issuebody[0]) , str(issuecomments[0]) ,str(isTrain)]
            tup_line.append(tup)
        else:
             print("Issue:",issue, " repeated:", pr)   

    else:
        print("Issue:",issue, " was in fact an PR! pr base:", pr)
        
    i=i+1
    pr_ant = pr
    issue_ant = issue
    
    
#newFields2 = pd.DataFrame.from_records(data=[tup_line], columns=[headerdf])
newFields3 = pd.DataFrame(tup_line)  
 
newFields = pd.DataFrame(line)
newFields3.columns = ['PR', 'Issue', 'Issue_Title_Linked', 'Issue_Body_Linked', 'Issue_Comment_Linked', 'isTrain']

newFields.to_csv( new_fields_csv, encoding='utf-8', header=True, index=False, sep=',' )
np.savetxt( new_fields_txt, newFields.values, fmt='%s', delimiter=',' )

newFields3.to_csv( new_fields3_csv, encoding='utf-8', header=True, index=False, sep=',' )
np.savetxt( new_fields3_txt, newFields3.values, fmt='%s', delimiter=',')


processing... 841
Issue: 841  was in fact an PR! pr base: 7
processing... 869
Issue: 869  was in fact an PR! pr base: 8
processing... 880
Issue: 880  linked from PR: 11
processing... 959
Issue: 959  linked from PR: 12
processing... 959
Issue: 959  repeated: 12
processing... 960
Issue: 960  linked from PR: 14
processing... 960
Issue: 960  repeated: 14
processing... 850
Issue: 850  linked from PR: 17
processing... 958
Issue: 958  linked from PR: 26
processing... 762
Issue: 762  linked from PR: 28
processing... 692
Issue: 692  was in fact an PR! pr base: 28
processing... 685
Issue: 685  linked from PR: 28
processing... 602
Issue: 602  was in fact an PR! pr base: 28
processing... 225
Issue: 225  was in fact an PR! pr base: 28
processing... 1240
Issue: 1240  was in fact an PR! pr base: 29
processing... 1240
Issue: 1240  was in fact an PR! pr base: 29
processing... 1253
Issue: 1253  linked from PR: 36
processing... 919
Issue: 919  linked from PR: 38
processing... 1269
Issue: 1269  was in fac

Issue: 124  linked from PR: 1246
processing... 1245
Issue: 1245  linked from PR: 1246
processing... 473
Issue: 473  linked from PR: 1269
processing... 1270
Issue: 1270  linked from PR: 1277
processing... 1251
Issue: 1251  linked from PR: 1278
processing... 625
Issue: 625  linked from PR: 1281
processing... 1271
Issue: 1271  linked from PR: 1282
processing... 1281
Issue: 1281  was in fact an PR! pr base: 1296
processing... 871
Issue: 871  linked from PR: 1298
processing... 1290
Issue: 1290  linked from PR: 1300
processing... 1294
Issue: 1294  linked from PR: 1304
processing... 1295
Issue: 1295  linked from PR: 1310
processing... 1312
Issue: 1312  linked from PR: 1318
processing... 132
Issue: 132  linked from PR: 1326
processing... 1324
Issue: 1324  linked from PR: 1336
processing... 134
Issue: 134  linked from PR: 1344
processing... 1347
Issue: 1347  linked from PR: 1349
processing... 1294
Issue: 1294  linked from PR: 1350
processing... 1294
Issue: 1294  repeated: 1350
processing... 135

Issue: 2054  linked from PR: 2055
processing... 2060
Issue: 2060  linked from PR: 2061
processing... 2012
Issue: 2012  linked from PR: 2066
processing... 200
Issue: 200  was in fact an PR! pr base: 2067
processing... 2002
Issue: 2002  was in fact an PR! pr base: 2067
processing... 2064
Issue: 2064  linked from PR: 2067
processing... 1808
Issue: 1808  linked from PR: 2069
processing... 1704
Issue: 1704  was in fact an PR! pr base: 2070
processing... 2021
Issue: 2021  linked from PR: 2070
processing... 2064
Issue: 2064  linked from PR: 2075
processing... 1989
Issue: 1989  was in fact an PR! pr base: 2077
processing... 1886
Issue: 1886  linked from PR: 2082
processing... 1989
Issue: 1989  was in fact an PR! pr base: 2088
processing... 2078
Issue: 2078  linked from PR: 2088
processing... 2090
Issue: 2090  linked from PR: 2093
processing... 2021
Issue: 2021  linked from PR: 2094
processing... 1993
Issue: 1993  linked from PR: 2102
processing... 2091
Issue: 2091  linked from PR: 2102
process

<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :


2102
processing... 2104
Issue: 2104  linked from PR: 2106
processing... 2110
Issue: 2110  linked from PR: 2113
processing... 2109
Issue: 2109  linked from PR: 2114
processing... 2050
Issue: 2050  linked from PR: 2121
processing... 2105
Issue: 2105  linked from PR: 2125
processing... 2142
Issue: 2142  linked from PR: 2129
processing... 167
Issue: 167  linked from PR: 2145
processing... 29
Issue: 29  was in fact an PR! pr base: 2152
processing... 97
Issue: 97  linked from PR: 2153
processing... 2119
Issue: 2119  linked from PR: 2161
processing... 2098
Issue: 2098  linked from PR: 2161
processing... 1964
Issue: 1964  linked from PR: 2162
processing... 1964
Issue: 1964  repeated: 2162
processing... 2133
Issue: 2133  linked from PR: 2163
processing... 216
Issue: 216  linked from PR: 2169
processing... 2166
Issue: 2166  linked from PR: 2175
processing... 2177
Issue: 2177  linked from PR: 2179
processing... 2184
Issue: 2184  linked from PR: 2196
processing... 2186
Issue: 2186  was in fact an 

...

7220
processing... 468
Issue: 468  was in fact an PR! pr base: 7220
processing... 422
Issue: 422  was in fact an PR! pr base: 7220
processing... 478
Issue: 478  was in fact an PR! pr base: 7220
processing... 479
Issue: 479  linked from PR: 7220
processing... 486
Issue: 486  was in fact an PR! pr base: 7220
processing... 401
Issue: 401  was in fact an PR! pr base: 7220
processing... 484
Issue: 484  was in fact an PR! pr base: 7220
processing... 7200
Issue: 7200  linked from PR: 7224
processing... 6369
Issue: 6369  linked from PR: 7225
processing... 5833
Issue: 5833  linked from PR: 7228
processing... 5804
Issue: 5804  linked from PR: 7229
processing... 7075
Issue: 7075  was in fact an PR! pr base: 7229
processing... 6334
Issue: 6334  was in fact an PR! pr base: 7231
processing... 6334
Issue: 6334  was in fact an PR! pr base: 7231
processing... 7197
Issue: 7197  linked from PR: 7232
processing... 7237
Issue: 7237  linked from PR: 7240
processing... 7239
Issue: 7239  linked from PR: 7243


Issue: 499  linked from PR: 7463
processing... 539
Issue: 539  linked from PR: 7463
processing... 525
Issue: 525  was in fact an PR! pr base: 7463
processing... 540
Issue: 540  was in fact an PR! pr base: 7463
processing... 539
Issue: 539  linked from PR: 7463
processing... 538
Issue: 538  was in fact an PR! pr base: 7463
processing... 7464
Issue: 7464  linked from PR: 7470
processing... 702
Issue: 702  linked from PR: 7471
processing... 7020
Issue: 7020  linked from PR: 7471
processing... 7452
Issue: 7452  linked from PR: 7474
processing... 9272
Issue: 9272  was in fact an PR! pr base: 7475
processing... 9251
Issue: 9251  was in fact an PR! pr base: 7475
processing... 7655
Issue: 7655  was in fact an PR! pr base: 7475
processing... 9317
Issue: 9317  was in fact an PR! pr base: 7475
processing... 9304
Issue: 9304  was in fact an PR! pr base: 7475
processing... 81
Issue: 81  was in fact an PR! pr base: 7476
processing... 81
Issue: 81  was in fact an PR! pr base: 7476
processing... 82
Is

<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: Depr

 7692
processing... 117
Issue: 117  linked from PR: 7692
processing... 117
Issue: 117  repeated: 7692
processing... 6207
Issue: 6207  linked from PR: 7693
processing... 6207
Issue: 6207  linked from PR: 7694
processing... 6207
Issue: 6207  linked from PR: 7696
processing... 6207
Issue: 6207  linked from PR: 7697
processing... 6207
Issue: 6207  linked from PR: 7698
processing... 6207
Issue: 6207  linked from PR: 7700
processing... 6197
Issue: 6197  linked from PR: 7702
processing... 7346
Issue: 7346  linked from PR: 7705
processing... 7439
Issue: 7439  linked from PR: 7705
processing... 7395
Issue: 7395  linked from PR: 7708
processing... 6966
Issue: 6966  linked from PR: 7709
processing... 7710
Issue: 7710  linked from PR: 7711
processing... 7710
Issue: 7710  repeated: 7711
processing... 4682
Issue: 4682  was in fact an PR! pr base: 7714
processing... 7716
Issue: 7716  linked from PR: 7717
processing... 7451
Issue: 7451  linked from PR: 7726
processing... 7451
Issue: 7451  repeated: 77

<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (ispr==0) :
<ipython-input-22-ce03486be8e1>:22: Depr

In [23]:
type(ispr)

numpy.ndarray

In [24]:
type(issue)

str

In [25]:
ispr


array([0])

In [26]:
ispr = prIssues.loc[prIssues['Issue_Number'] == 959, 'isPR'].values[0]

In [27]:
ispr


0

In [28]:
prIssues.loc[prIssues['Issue_Number'] == 959]

,Issue_Number,Issue_Title,Issue_Author_Name,Issue_Author_Login,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author_Name,PR_Author_Login,PR_Closed_Date,PR_Body,PR_Comments,Commit_Author_Name,Commit_Date,Commit_Message,isPR
956,959,"Cleanup entries: Rely on ""field formatter""",Oliver Kopp,koppor,"05/12/16, 08:02:09 PM","The first two cleanups can be achieved using the ""field formatters"".\n- [ ] ""Run Unico...",All the points are already fixed in the latest master version.\n =||=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [29]:
issue

'7786'

In [30]:
int(issue)

7786

In [31]:
issuetitle[0]

'Library import: no "select all" option'

In [32]:
issuebody[0]

'JabRef 5.2--2020-12-24--6a2a512\r\nWindows 10 10.0 amd64 \r\nJava 14.0.2\r\n\r\nWhen importing another library into the current one, a dialog opens in which you have to select entries.\r\nThere is a "select new entries", which is not reliable (I tried with existing entries that had been modified: the software does not see them as new), and a "unselect all".\r\n\r\nWhy not add a "select all"?\r\n\r\nCtrl-click and shift-click do not even provide this functionality.\r\n\r\nMy production rate has definitly lowered after my third bug report this day'

In [33]:
issuecomments[0]

'Thanks for your suggestion. Can you please elaborate on why you want to import duplicate entries? If you later want to merge them with an existing entry, this is already possible in the import dialog: you have a small icon next to the duplicated item that lets you manage the import / merge strategy for this entry. =||= I have the same issiue on Windows 10. I cannot import PDF files at once. Drag and Drop doenst work.. How to add multiple PDF files to library ?\r\n\r\nEdit: Drag and Drop work on version 5.3 =||= Thanks to @brapana  this issue is now resolved int he latest main!  =||= '

In [34]:
newFields = pd.read_csv( new_fields_csv, header = 0, sep=",")

In [35]:
newFields3 = pd.read_csv( new_fields3_csv, header = 0, sep=",")

In [36]:
newFields3.head(10)

,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
0,11.0,880,OO/LO: Jabref shoud respect MaxAuthors/MaxFirstAuthors in Bibliography,"Jab Ref 3.2 Development version, Libre Office 5.0.2 x64 on Windows 7x63\n\nWhen I inse...",Are those lengths always the same or should one add another parameter for the bibliogr...,0.0
1,12.0,959,"Cleanup entries: Rely on ""field formatter""","The first two cleanups can be achieved using the ""field formatters"".\n- [ ] ""Run Unico...",All the points are already fixed in the latest master version.\n =||=,0.0
2,14.0,960,Remove trailing ; at metadata,"While reading test cases of the metadata serialization, I saw that the final `;` seems...","No objections, but we still need to be able to parse the old format. But we can write ...",0.0
3,17.0,850,Remove private fields (and other field flags),Private fields (fields marked with `net.sf.jabref.gui.InternalBibtexFields.BibtexSingl...,"@JabRef/developers Are these fields removed? I cannot find anything anymore, when sear...",0.0
4,26.0,958,Size of database properties dialog,The size of the database properties dialog is not fixed. When opening the dialog the f...,I did not notice a problem on Win7. The only thing is that the space between the last ...,0.0
5,28.0,762,[WIP] Replaced BibEntry[] with List<BibEntry>,Replaced `BibEntry[]` with `List<BibEntry>` as return from `MainTable.getSelectedEntri...,More or less the same changes as in #718 :crying_cat_face: \n =||= Oh no...\n =||= Wai...,0.0
6,28.0,685,Can't export to MySQL 5.5 in JabRef 3.2,"I'm using Jabref 3.2 and MySQL 5.5, but now I can't export my .bib to SQL database. It...","Is it possible to post a [minimal, complete, and verifiable example](https://en.wikipe...",0.0
7,36.0,1253,NullPointerException on editing bibtex file,JabRef version 2.10 on gentoo Linux\n\nSteps to reproduce: This recently started happe...,"Would you please test with the actual version 3.3?\n =||= Hi There,\n\nGentoo doesn't ...",0.0
8,38.0,919,Integrate journal abbreviations into JabRef,"Currently, the repository https://github.com/JabRef/reference-abbreviations contains s...","As a big fan of using strings for journal abbreviations, I also believe that it would ...",0.0
9,137.0,136,Extract and unify all GUI actions,The question is how can we reduce the cluttering caused by those inline classes.\n\nTh...,> The question is how can we reduce the cluttering caused by those inline classes.\n\n...,0.0


In [37]:
newFields3.columns = ['PR', 'Issue', 'Issue_Title_Linked', 'Issue_Body_Linked', 'Issue_Comment_Linked', 'isTrain']

In [38]:
newFields3.head(10)

,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
0,11.0,880,OO/LO: Jabref shoud respect MaxAuthors/MaxFirstAuthors in Bibliography,"Jab Ref 3.2 Development version, Libre Office 5.0.2 x64 on Windows 7x63\n\nWhen I inse...",Are those lengths always the same or should one add another parameter for the bibliogr...,0.0
1,12.0,959,"Cleanup entries: Rely on ""field formatter""","The first two cleanups can be achieved using the ""field formatters"".\n- [ ] ""Run Unico...",All the points are already fixed in the latest master version.\n =||=,0.0
2,14.0,960,Remove trailing ; at metadata,"While reading test cases of the metadata serialization, I saw that the final `;` seems...","No objections, but we still need to be able to parse the old format. But we can write ...",0.0
3,17.0,850,Remove private fields (and other field flags),Private fields (fields marked with `net.sf.jabref.gui.InternalBibtexFields.BibtexSingl...,"@JabRef/developers Are these fields removed? I cannot find anything anymore, when sear...",0.0
4,26.0,958,Size of database properties dialog,The size of the database properties dialog is not fixed. When opening the dialog the f...,I did not notice a problem on Win7. The only thing is that the space between the last ...,0.0
5,28.0,762,[WIP] Replaced BibEntry[] with List<BibEntry>,Replaced `BibEntry[]` with `List<BibEntry>` as return from `MainTable.getSelectedEntri...,More or less the same changes as in #718 :crying_cat_face: \n =||= Oh no...\n =||= Wai...,0.0
6,28.0,685,Can't export to MySQL 5.5 in JabRef 3.2,"I'm using Jabref 3.2 and MySQL 5.5, but now I can't export my .bib to SQL database. It...","Is it possible to post a [minimal, complete, and verifiable example](https://en.wikipe...",0.0
7,36.0,1253,NullPointerException on editing bibtex file,JabRef version 2.10 on gentoo Linux\n\nSteps to reproduce: This recently started happe...,"Would you please test with the actual version 3.3?\n =||= Hi There,\n\nGentoo doesn't ...",0.0
8,38.0,919,Integrate journal abbreviations into JabRef,"Currently, the repository https://github.com/JabRef/reference-abbreviations contains s...","As a big fan of using strings for journal abbreviations, I also believe that it would ...",0.0
9,137.0,136,Extract and unify all GUI actions,The question is how can we reduce the cluttering caused by those inline classes.\n\nTh...,> The question is how can we reduce the cluttering caused by those inline classes.\n\n...,0.0


In [39]:
newFields.head(10)

,0
0,"PR, Issue, Issue_Title_Linked, Issue_Body_Linked, Issue_Comment_Linked, isTrain"
1,"11,880,OO/LO: Jabref shoud respect MaxAuthors/MaxFirstAuthors in Bibliography,Jab Ref ..."
2,"12,959,Cleanup entries: Rely on ""field formatter"",The first two cleanups can be achiev..."
3,"14,960,Remove trailing ; at metadata,While reading test cases of the metadata serializ..."
4,"17,850,Remove private fields (and other field flags),Private fields (fields marked wit..."
5,"26,958,Size of database properties dialog,The size of the database properties dialog i..."
6,"28,762,[WIP] Replaced BibEntry[] with List<BibEntry>,Replaced `BibEntry[]` with `List<..."
7,"28,685,Can't export to MySQL 5.5 in JabRef 3.2,I'm using Jabref 3.2 and MySQL 5.5, but..."
8,"36,1253,NullPointerException on editing bibtex file,JabRef version 2.10 on gentoo Linu..."
9,"38,919,Integrate journal abbreviations into JabRef,Currently, the repository https://g..."


In [40]:
df_merge = pd.merge(prIssues, newFields3, left_on='Issue_Number', right_on='PR', how='left')

In [41]:
df_merge.head(15)

,Issue_Number,Issue_Title,Issue_Author_Name,Issue_Author_Login,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author_Name,PR_Author_Login,...,Commit_Author_Name,Commit_Date,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
0,1,New Sorting/Export preferences,Olaf Lenz,olenz,"03/12/14, 11:38:01 AM","This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,Olaf Lenz,olenz,...,Olaf Lenz,"03/12/14, 09:16:45 AM",Merge branch 'sorting',1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Basic gradle integration,Simon Harrer,simonharrer,"03/12/14, 06:29:22 PM",This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Basic gradle integration,Simon Harrer,simonharrer,...,Simon Harrer,"03/12/14, 05:20:08 PM",Gradle build works basically. Reuses existing folder structure.,1,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,egonw,"03/15/14, 12:55:32 PM",NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,egonw,...,Egon Willighagen,"03/15/14, 09:33:10 AM",Some example new Dutch translations via the GitHub web page.,1,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Spanish translation update,Jorge Tornero,jtornero,"03/17/14, 10:20:50 PM",Three new strings translated.,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,Jorge Tornero,jtornero,...,jtornero,"03/17/14, 09:28:26 PM",Spanish translation update,1,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Update JabRef_in.properties,NaN,was123,"03/18/14, 05:12:53 AM",Indonesian translation added,NaN,Update JabRef_in.properties,NaN,was123,...,was123,"03/18/14, 12:01:58 AM",Update JabRef_in.properties\n\nIndonesian translation added,1,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Fixed splash-dev.svg,Olaf Lenz,olenz,"03/19/14, 07:46:25 PM",NaN,NaN,Fixed splash-dev.svg,Olaf Lenz,olenz,...,Olaf Lenz,"03/19/14, 07:41:29 PM",Fixed splash-dev.svg,1,NaN,NaN,NaN,NaN,NaN,NaN
6,7,I have implemented 2 features requested in Ticket #841.,NaN,noravanq,"03/23/14, 04:03:12 PM","```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN,I have implemented 2 features requested in Ticket #841.,NaN,noravanq,...,noravanq,"03/23/14, 04:48:49 AM",I have implemented 2 features requested in Ticket #841.\n\n The user can now speci...,1,NaN,NaN,NaN,NaN,NaN,NaN
7,8,PDF-file metadata: Privacy Filtering all metadata,Adrian Daerr,adaerr,"04/23/14, 09:23:03 PM",This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n =||=,PDF-file metadata: Privacy Filtering all metadata,Adrian Daerr,adaerr,...,Adrian Daerr,"04/22/14, 06:19:15 PM",Erase fields listed in XMP Privacy Settings from PDF DocumentInformation when XMP-tagging,1,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Support FindFullText with ACS DOIs,Peter Ansell,ansell,"05/20/14, 12:53:43 PM",Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN,Support FindFullText with ACS DOIs,Peter Ansell,ansell,...,Peter Ansell,"05/07/14, 01:02:09 AM",add CSIRO copyright 2014,1,NaN,NaN,NaN,NaN,NaN,NaN
9,10,try to fix some obvious bugs about `groups`,NaN,braindevices,"05/20/14, 12:55:20 PM","Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...,try to fix some obvious bugs about `groups`,NaN,braindevices,...,Ling Wang,"05/20/14, 02:56:41 AM",fix bug causing double entries.,1,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_merge['isTrain'] = df_merge['isTrain'].replace(np.nan, 1)

In [43]:
df_merge.head()

,Issue_Number,Issue_Title,Issue_Author_Name,Issue_Author_Login,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author_Name,PR_Author_Login,...,Commit_Author_Name,Commit_Date,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
0,1,New Sorting/Export preferences,Olaf Lenz,olenz,"03/12/14, 11:38:01 AM","This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,Olaf Lenz,olenz,...,Olaf Lenz,"03/12/14, 09:16:45 AM",Merge branch 'sorting',1,NaN,NaN,NaN,NaN,NaN,1.0
1,2,Basic gradle integration,Simon Harrer,simonharrer,"03/12/14, 06:29:22 PM",This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN,Basic gradle integration,Simon Harrer,simonharrer,...,Simon Harrer,"03/12/14, 05:20:08 PM",Gradle build works basically. Reuses existing folder structure.,1,NaN,NaN,NaN,NaN,NaN,1.0
2,3,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,egonw,"03/15/14, 12:55:32 PM",NaN,NaN,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,egonw,...,Egon Willighagen,"03/15/14, 09:33:10 AM",Some example new Dutch translations via the GitHub web page.,1,NaN,NaN,NaN,NaN,NaN,1.0
3,4,Spanish translation update,Jorge Tornero,jtornero,"03/17/14, 10:20:50 PM",Three new strings translated.,"Even though the other translation files have been updated, I accept this pull request....",Spanish translation update,Jorge Tornero,jtornero,...,jtornero,"03/17/14, 09:28:26 PM",Spanish translation update,1,NaN,NaN,NaN,NaN,NaN,1.0
4,5,Update JabRef_in.properties,NaN,was123,"03/18/14, 05:12:53 AM",Indonesian translation added,NaN,Update JabRef_in.properties,NaN,was123,...,was123,"03/18/14, 12:01:58 AM",Update JabRef_in.properties\n\nIndonesian translation added,1,NaN,NaN,NaN,NaN,NaN,1.0


In [44]:
df_merge.to_csv( merge_output_filename, encoding='utf-8', header=True, index=False, sep=',') 

In [45]:
df_merge.to_csv( mergeNA_output_filename, encoding='utf-8', header=True, index=False, na_rep='NA', sep=',') 

In [46]:
newFields3.loc[newFields3['PR'] == 4113]

,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain


In [47]:
df_merge.loc[df_merge['Issue_Number'] == 2300]


,Issue_Number,Issue_Title,Issue_Author_Name,Issue_Author_Login,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author_Name,PR_Author_Login,...,Commit_Author_Name,Commit_Date,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
2353,2300,Fix local metadata synchronization (fixes: #2219),Admir Obralija,obraliar,"11/20/16, 06:19:25 PM",Issue: https://github.com/JabRef/jabref/issues/2219.\r\n\r\nThis issue is fixed by ove...,"I merged it, because it is a hotfix. @obraliar Do you think, it is possible to add a t...",Fix local metadata synchronization (fixes: #2219),Admir Obralija,obraliar,...,Admir Obralija,"11/20/16, 05:21:07 PM",Fix local metadata synchronization\n\n- Overload the parse method in MetaDataParser\n-...,1,2300.0,2219,Metadata lost on reconnect to a shared database,JabRef 3.7dev--snapshot--2016-10-24--master--7d6ee2b\r\nlinux 3.13.0-100-generic i386 ...,What is the status of this issue? @tobiasdiez @obraliar \n =||= I'm unfortunately not ...,0.0


In [48]:
prIssues.loc[prIssues['Issue_Number'] == 1, 'PR_Body']


0    This will add a new "File Sorting" Tab to the preferences that allows to exactly speci...
Name: PR_Body, dtype: object

In [49]:
print( list( df_merge ))

['Issue_Number', 'Issue_Title', 'Issue_Author_Name', 'Issue_Author_Login', 'Issue_Closed_Date', 'Issue_Body', 'Issue_Comments', 'PR_Title', 'PR_Author_Name', 'PR_Author_Login', 'PR_Closed_Date', 'PR_Body', 'PR_Comments', 'Commit_Author_Name', 'Commit_Date', 'Commit_Message', 'isPR', 'PR', 'Issue', 'Issue_Title_Linked', 'Issue_Body_Linked', 'Issue_Comment_Linked', 'isTrain']


In [50]:
value = df_merge['PR_Title'].str.replace(",",'')
df_merge['PR_Title'] = value

value = df_merge['PR_Body'].str.replace(",",'')
df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace(",",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace(",",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace(",",'')
df_merge['Issue_Comments'] = value

print( value )

#value = df_merge['PR_Comments'].str.replace(",",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace(",",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace(",",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace(",",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace(",",'')
df_merge['Issue_Comment_Linked'] = value


0                                                                                             NaN
1                                                                                             NaN
2                                                                                             NaN
3       Even though the other translation files have been updated I accept this pull request.\...
4                                                                                             NaN
5                                                                                             NaN
6                                                                                             NaN
7                                                                                    thx!\n =||= 
8                                                                                             NaN
9       After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...
10                  

In [51]:
value = df_merge['PR_Title'].str.replace("\n",'')
df_merge['PR_Title'] = value

value = df_merge['PR_Body'].str.replace("\n",'')
df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace("\n",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace("\n",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace("\n",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace("\n",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace("\n",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace("\n",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace("\n",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace("\n",'')
df_merge['Issue_Comment_Linked'] = value


In [52]:
value = df_merge['PR_Title'].str.replace("'",'')
df_merge['PR_Title'] = value

value = df_merge['PR_Body'].str.replace("'",'')
df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace("'",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace("'",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace("'",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace("'",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace("'",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace("'",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace("'",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace("'",'')
df_merge['Issue_Comment_Linked'] = value


In [53]:
#value = df_merge['PR_Title'].str.replace("\r",'')
#df_merge['PR_Title'] = value

#value = df_merge['PR_Body'].str.replace("\r",'')
#df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace("\r",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace("\r",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace("\r",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace("\r",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace("\r",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace("\r",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace("\r",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace("\r",'')
df_merge['Issue_Comment_Linked'] = value


In [54]:
#value = df_merge['PR_Title'].str.replace(".",'')
#df_merge['PR_Title'] = value

#value = df_merge['PR_Body'].str.replace(".",'')
#df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace(".",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace(".",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace(".",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace(".",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace(".",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace(".",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace(".",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace(".",'')
df_merge['Issue_Comment_Linked'] = value


<ipython-input-54-4d0e4b42e6ae>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  value = df_merge['Issue_Title'].str.replace(".",'')
<ipython-input-54-4d0e4b42e6ae>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  value = df_merge['Issue_Body'].str.replace(".",'')
<ipython-input-54-4d0e4b42e6ae>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  value = df_merge['Issue_Comments'].str.replace(".",'')
<ipython-input-54-4d0e4b42e6ae>:19: FutureWarning: The default value of regex will change from True to False in a future versi

In [55]:
#value = df_merge['PR_Title'].str.replace(";",'')
#df_merge['PR_Title'] = value

#value = df_merge['PR_Body'].str.replace(";",'')
#df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace(";",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace(";",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace(";",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace(";",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace(";",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace(";",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace(";",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace(";",'')
df_merge['Issue_Comment_Linked'] = value

In [56]:
#value = df_merge['PR_Title'].str.replace("=||=",'')
#df_merge['PR_Title'] = value

#value = df_merge['PR_Body'].str.replace("=||=",'')
#df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace("=||=",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace("=||=",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace("=||=",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace("=||=",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace("=||=",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace("=||=",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace("=||=",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace("=||=",'')
df_merge['Issue_Comment_Linked'] = value

<ipython-input-56-1a34c2675138>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Title'].str.replace("=||=",'')
<ipython-input-56-1a34c2675138>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Body'].str.replace("=||=",'')
<ipython-input-56-1a34c2675138>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Comments'].str.replace("=||=",'')
<ipython-input-56-1a34c2675138>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Commit_Message'].str.replace("=||=",'')
<ipython-input-56-1a34c2675138>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Title_Linked'].str.replace("=||=",'')
<ipython-input-56-1a34c2675138>:25: FutureWarning: The

In [57]:
#value = df_merge['PR_Title'].str.replace("|=|",'')
#df_merge['PR_Title'] = value

#value = df_merge['PR_Body'].str.replace("|=|",'')
#df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace("|=|",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace("|=|",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace("|=|",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace("|=|",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace("|=|",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace("|=|",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace("|=|",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace("|=|",'')
df_merge['Issue_Comment_Linked'] = value

<ipython-input-57-6fdfe902898e>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Title'].str.replace("|=|",'')
<ipython-input-57-6fdfe902898e>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Body'].str.replace("|=|",'')
<ipython-input-57-6fdfe902898e>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Comments'].str.replace("|=|",'')
<ipython-input-57-6fdfe902898e>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Commit_Message'].str.replace("|=|",'')
<ipython-input-57-6fdfe902898e>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Title_Linked'].str.replace("|=|",'')
<ipython-input-57-6fdfe902898e>:25: FutureWarning: The defa

In [58]:
#value = df_merge['PR_Title'].str.replace("||",'')
#df_merge['PR_Title'] = value

#value = df_merge['PR_Body'].str.replace("||",'')
#df_merge['PR_Body'] = value

value = df_merge['Issue_Title'].str.replace("||",'')
df_merge['Issue_Title'] = value

value = df_merge['Issue_Body'].str.replace("||",'')
df_merge['Issue_Body'] = value

value = df_merge['Issue_Comments'].str.replace("||",'')
df_merge['Issue_Comments'] = value

#value = df_merge['PR_Comments'].str.replace("||",'')
#df_merge['PR_Comments'] = value

value = df_merge['Commit_Message'].str.replace("||",'')
df_merge['Commit_Message'] = value

value = df_merge['Issue_Title_Linked'].str.replace("||",'')
df_merge['Issue_Title_Linked'] = value

value = df_merge['Issue_Body_Linked'].str.replace("||",'')
df_merge['Issue_Body_Linked'] = value

value = df_merge['Issue_Comment_Linked'].str.replace("||",'')
df_merge['Issue_Comment_Linked'] = value

<ipython-input-58-766f86a77735>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Title'].str.replace("||",'')
<ipython-input-58-766f86a77735>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Body'].str.replace("||",'')
<ipython-input-58-766f86a77735>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Comments'].str.replace("||",'')
<ipython-input-58-766f86a77735>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Commit_Message'].str.replace("||",'')
<ipython-input-58-766f86a77735>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  value = df_merge['Issue_Title_Linked'].str.replace("||",'')
<ipython-input-58-766f86a77735>:25: FutureWarning: The default v

In [59]:
pd.options.display.max_colwidth = 900

In [60]:
df_merge.loc[df_merge['Issue_Number'] == 110]

,Issue_Number,Issue_Title,Issue_Author_Name,Issue_Author_Login,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author_Name,PR_Author_Login,...,Commit_Author_Name,Commit_Date,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
110,110,"Extract libraries out of JabRef (""Toolkit"") / Offer JabRef as library / jar / maven central",Oliver Kopp,koppor,"08/24/19, 10:57:43 PM",[Bioclipse](http://wwwbioclipsenet/) has used JabRef in the past for the BibTeX data model and there has been talk for reusing JabRef in PathVisio too for working with biobliograph data do you see any chance of splitting out a JabRef functionality library with core functionality (data model look up of data based on DOI or PubMed ID )?Source [message 32103878](https://sourceforgenet/p/jabref/mailman/message/32103878/) from @egonw- [ ] [orgjabrefmodel](https://githubcom/JabRef/jabref/tree/master/src/main/java/org/jabref/model)- [ ] [orgjabreflogic](https://githubcom/JabRef/jabref/tree/master/src/main/java/org/jabref/logic)- [ ] XMPUtil (see #266)- [ ] `netsfjabrefbstVM` (see https://githubcom/JabRef/jabref/issues/119#issuecomment-152326777) - refs https://githubcom/JabRef/jabref/pull/5702- [ ] many more (to be defined),Yes I can see a chance for this Especially because of all the cleanup and class sorting we are doing right now The data model is now placed into the `model` package and core functionality is moved into `logic` These two packages could be turned into a JabRef functionality library JabRef itself could benefit a lot from this separation of concerns Unfortunately there is still quite some way to go || As of 86f635c855b83ee94230a72afcab15c7f3cd51e8 the dependencies of the model package are limited to logic classes classes from the Java API and the infamous global classes (`Globals` `JabRefPreferences`) This is one step towards extracting a library || @lenhard Didnt you remove the dependency on Globals in `BibtexEntryType` before?! It breaks my tests as I now? need an instance of `JabRefPreferences` in any test || You mean BibtexEntryType right?In that case no I did not Neither now nor bef...,NaN,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1.0


In [61]:
print( list( df_merge ))

['Issue_Number', 'Issue_Title', 'Issue_Author_Name', 'Issue_Author_Login', 'Issue_Closed_Date', 'Issue_Body', 'Issue_Comments', 'PR_Title', 'PR_Author_Name', 'PR_Author_Login', 'PR_Closed_Date', 'PR_Body', 'PR_Comments', 'Commit_Author_Name', 'Commit_Date', 'Commit_Message', 'isPR', 'PR', 'Issue', 'Issue_Title_Linked', 'Issue_Body_Linked', 'Issue_Comment_Linked', 'isTrain']


In [62]:
#df_merge = df_merge.drop('Issue_Closed_Date', 1)
#df_merge = df_merge.drop('Issue_Author_Name', 1)
#df_merge = df_merge.drop('PR_Closed_Date', 1)
#df_merge = df_merge.drop('PR_Author_Name', 1)
#df_merge = df_merge.drop('PR_Title', 1)
#df_merge = df_merge.drop('PR_Body', 1)
#df_merge = df_merge.drop('Commit_Author_Name', 1)
#df_merge = df_merge.drop('Commit_Date', 1)

del df_merge['Issue_Closed_Date']
del df_merge['Issue_Author_Name']
del df_merge['PR_Closed_Date']
del df_merge['PR_Author_Name']
del df_merge['PR_Title']
del df_merge['PR_Body']
del df_merge['Commit_Author_Name']
del df_merge['Commit_Date']

In [63]:
df_merge.head()

,Issue_Number,Issue_Title,Issue_Author_Login,Issue_Body,Issue_Comments,PR_Author_Login,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
0,1,New Sorting/Export preferences,olenz,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly specify the sort and export order of entries rather than the rather limited set of choices that were available to date",NaN,olenz,NaN,Merge branch sorting,1,NaN,NaN,NaN,NaN,NaN,1.0
1,2,Basic gradle integration,simonharrer,This adds basic gradle integration The project files for Intellij and Eclipse can be generated and JabRef can be started from within the IDEs as well as the tests can be run with errors,NaN,simonharrer,NaN,Gradle build works basically Reuses existing folder structure,1,NaN,NaN,NaN,NaN,NaN,1.0
2,3,Some example new Dutch translations via the GitHub web page,egonw,NaN,NaN,egonw,NaN,Some example new Dutch translations via the GitHub web page,1,NaN,NaN,NaN,NaN,NaN,1.0
3,4,Spanish translation update,jtornero,Three new strings translated,"Even though the other translation files have been updated I accept this pull requestFor next time please try to commit only the ""really"" changed file and revert the changes of the other one You can also use the edit button at https://githubcom/JabRef/jabref/blob/master/src/resource/JabRef_esproperties ||",jtornero,NaN,Spanish translation update,1,NaN,NaN,NaN,NaN,NaN,1.0
4,5,Update JabRef_inproperties,was123,Indonesian translation added,NaN,was123,NaN,Update JabRef_inpropertiesIndonesian translation added,1,NaN,NaN,NaN,NaN,NaN,1.0


In [64]:
df_merge.to_csv( mergeNA_output_filename, encoding='utf-8', header=True, index=False, na_rep='NA', sep=',') 

In [65]:
df_merge.to_csv( merge_output_filename, encoding='utf-8', header=True, index=False, sep=',') 